# Natural Language Processing

In [3]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F

import random, math, time
import pandas as pd
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


## 1. ETL: Loading the dataset

**Note**: Here I chose to translate English to German, simply it is easier for myself, since I don't understand German so it is difficult for me to imagine a sentence during inference.

In [11]:



df = pd.read_csv('./datset_translate_train/generated_reviews_crowd.csv')
df = df.reindex(columns= ["th_text","en_text"])
df.head()
dataset = df.values.tolist()
dataset_size = len(dataset)

random.shuffle(dataset)
train_data_set_size = int(dataset_size *0.8)
train_megaDataset = dataset[:train_data_set_size]
test_megaDset = dataset[train_data_set_size:]
random.shuffle(train_megaDataset)


train_size = int(train_data_set_size *0.1)
train = train_megaDataset[:train_size]



val = test_megaDset[train_size:100+train_size]

test = test_megaDset[100+train_size:200+train_size]

print('testSize=',len(test),'trainSize=',len(train),'valsize = ',len(val))

testSize= 100 trainSize= 1966 valsize =  100


## 3. Preprocessing 

### Tokenizing

In [13]:
from attacut import tokenize, Tokenizer

# tokenize `txt` using our best model `attacut-sc`
txt = 'ผมชื่อลุงเริง'
words = tokenize(txt)
print(words)
# alternatively, an AttaCut tokenizer might be instantiated directly, allowing
# one to specify whether to use `attacut-sc` or `attacut-c`.
atta = Tokenizer(model="attacut-sc")
words = atta.tokenize(txt)
words

['ผม', 'ชื่อ', 'ลุงเริง']


['ผม', 'ชื่อ', 'ลุงเริง']

In [14]:
SRC_LANGUAGE = 'th'
TRG_LANGUAGE = 'en'
# Place-holders
token_transform = {}
vocab_transform = {}

In [15]:
from attacut import tokenize, Tokenizer
from torchtext.data.utils import get_tokenizer
token_transform[SRC_LANGUAGE] = Tokenizer(model="attacut-sc")
token_transform[TRG_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

In [20]:
# # tokenizer
# #thai part
# print("Sentence= ", train[0][0])
# print("Tokenization= ", token_transform[SRC_LANGUAGE].tokenize(train[0][0]))
# # english part
# print("Sentence= ", train[0][1])
# print("Tokenizatio= ", token_transform[TRG_LANGUAGE](train[0][1]))

In [18]:

def yield_tokens(data, language):
    language_index = {SRC_LANGUAGE: 0, TRG_LANGUAGE: 1}

    for data_sample in data:
        if language == SRC_LANGUAGE:
            yield token_transform[language].tokenize(data_sample[language_index[language]])
        elif language == TRG_LANGUAGE:
            yield token_transform[language](data_sample[language_index[language]])

In [19]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

In [21]:
from torchtext.vocab import build_vocab_from_iterator

for ln in [SRC_LANGUAGE, TRG_LANGUAGE]:
    # Create torchtext's Vocab object 
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train, ln), 
                                                    min_freq=2,   #if not, everything will be treated as UNK
                                                    specials=special_symbols,
                                                    special_first=True) #indicates whether to insert symbols at the beginning or at the end                                            
# Set UNK_IDX as the default index. This index is returned when the token is not found. 
# If not set, it throws RuntimeError when the queried token is not found in the Vocabulary. 
for ln in [SRC_LANGUAGE, TRG_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [22]:
import pickle
with open('vocab_transform.pickle', 'wb') as handle:
    pickle.dump(vocab_transform, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
#see some example
vocab_transform[TRG_LANGUAGE](['here', 'is', 'a', 'unknownword', 'a'])

[219, 14, 12, 0, 12]

In [24]:
#we can reverse it....
mapping = vocab_transform[TRG_LANGUAGE].get_itos()
# #print 1816, for example
mapping[3]

'<eos>'

In [25]:
#we can reverse it....
mapping = vocab_transform[SRC_LANGUAGE].get_itos()
# #print 1816, for example
mapping[400]

'คะแนน'

In [26]:
#let's try unknown vocab
mapping[400]
#they will all map to <unk> which has 0 as integer

'คะแนน'

In [27]:
#let's try special symbols
mapping[1], mapping[2], mapping[3]

('<pad>', '<sos>', '<eos>')

In [28]:
#check unique vocabularies
len(mapping)

1298

## 4. Preparing the dataloader

One thing we change here is the <code>collate_fn</code> which now also returns the length of sentence.  This is required for <code>packed_padded_sequence</code>

In [29]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

BATCH_SIZE = 64

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            if transform == token_transform[SRC_LANGUAGE]:
                txt_input = transform.tokenize(txt_input)
            else:
                txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([SOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and trg language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TRG_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_batch(batch):
    src_batch, src_len_batch, trg_batch = [], [], []
    for src_sample, trg_sample in batch:
        processed_text = text_transform[SRC_LANGUAGE](src_sample.rstrip("\n"))
        src_batch.append(processed_text)
        trg_batch.append(text_transform[TRG_LANGUAGE](trg_sample.rstrip("\n")))
        src_len_batch.append(processed_text.size(0))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)
    return src_batch, torch.tensor(src_len_batch, dtype=torch.int64), trg_batch

Create train, val, and test dataloaders

In [30]:
batch_size = 10

train_loader = DataLoader(train, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(val, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test, batch_size=batch_size,
                             shuffle=True, collate_fn=collate_batch)

In [32]:
# import pickle
# with open('text_transform.pickle', 'wb') as handle:
#     pickle.dump(text_transform, handle, protocol=pickle.HIGHEST_PROTOCOL)

Let's test the train loader.

In [33]:
# for th, _, en in train_loader:
#     break

In [34]:
# print("Thai shape: ", th.shape)  # (seq len, batch_size)
# print("English shape: ", en.shape)   # (seq len, batch_size)

Thai shape:  torch.Size([45, 10])
English shape:  torch.Size([36, 10])


In [39]:
# train_loader_length = len(list(iter(train_loader)))
# val_loader_length   = len(list(iter(valid_loader)))
# test_loader_length  = len(list(iter(test_loader)))

In [41]:
train_loader_length = 197
val_loader_length = 10
test_loader_length = 10

In [42]:
train_loader_length, val_loader_length, test_loader_length  

(197, 10, 10)

## 5. Design the model

### Encoder

The changes here all within the `forward` method. It now accepts the lengths of the source sentences as well as the sentences themselves. 

After the source sentence (padded automatically within the iterator) has been embedded, we can then use `pack_padded_sequence` on it with the lengths of the sentences. Note that the tensor containing the lengths of the sequences must be a CPU tensor as of the latest version of PyTorch, which we explicitly do so with `to('cpu')`. `packed_embedded` will then be our packed padded sequence. This can be then fed to our RNN as normal which will return `packed_outputs`, a packed tensor containing all of the hidden states from the sequence, and `hidden` which is simply the final hidden state from our sequence. `hidden` is a standard tensor and not packed in any way, the only difference is that as the input was a packed sequence, this tensor is from the final **non-padded element** in the sequence.

We then unpack our `packed_outputs` using `pad_packed_sequence` which returns the `outputs` and the lengths of each, which we don't need. 

The first dimension of `outputs` is the padded sequence lengths however due to using a packed padded sequence the values of tensors when a padding token was the input will be all zeros.

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn       = nn.GRU(emb_dim, hid_dim, bidirectional=True)
        self.fc        = nn.Linear(hid_dim * 2, hid_dim)
        self.dropout   = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src: [src len, batch size]
        #src len: [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, 
                            src_len.to('cpu'), enforce_sorted=False)
        
        #packed_outputs contain all hidden states including padding guy
        #hidden contains the last hidden states of the non-padded guys
        packed_outputs, hidden = self.rnn(packed_embedded)
        #hidden: [n layers * num directions, batch size, hid dim]
        
        #convert packed_outputs to the guy that does not contain hidden states for padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        #outputs: [src len, batch size, hid dim * num directions]
        
        #take the last hidden states from backward and forward
        #hidden: (f, b, f, b)
        forward  = hidden[-2, :, :]  #[batch size, hid dim]
        backward = hidden[-1, :, :]  #[batch size, hid dim]
        
        hidden = torch.tanh(self.fc(torch.cat((forward, backward), dim = 1))) 
        #hidden: [batch size, hid dim]
        
        return outputs, hidden

### Attention

The attention used here is additive attention which is defined by:

$$e = v\text{tanh}(W_hh + W_ss + b)$$

Previously, we allowed this module to "pay attention" to padding tokens within the source sentence. However, using *masking*, we can force the attention to only be over non-padding elements.

The `forward` method now takes a `mask` input. This is a `[batch size, source sentence length]` tensor that is 1 when the source sentence token is not a padding token, and 0 when it is a padding token. For example, if the source sentence is: `["hello", "how", "are", "you", "?", `<pad>`, `<pad>`]`, then the mask would be `[1, 1, 1, 1, 1, 0, 0]`.

We apply the mask after the attention has been calculated, but before it has been normalized by the `softmax` function. It is applied using `masked_fill`. This fills the tensor at each element where the first argument (`mask == 0`) is true, with the value given by the second argument (`-1e10`). In other words, it will take the un-normalized attention values, and change the attention values over padded elements to be `-1e10`. As these numbers will be miniscule compared to the other values they will become zero when passed through the `softmax` layer, ensuring no attention is payed to padding tokens in the source sentence.

In [44]:
class Attention(nn.Module):
    def __init__(self, hid_dim, variants):
        super().__init__()
        self.variants = variants
        self.v = nn.Linear(hid_dim, 1, bias = False)
        self.W = nn.Linear(hid_dim,     hid_dim) #for decoder
        self.U = nn.Linear(hid_dim * 2, hid_dim) #for encoder outputs
                
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, hid dim * 2]

        if self.variants == 'additive': #work
            #repeat decoder hidden state src_len times
            hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
            #hidden = [batch size, src len, hid dim]
            
            energy = torch.tanh(self.W(hidden) + self.U(encoder_outputs))
            #energy = [batch size, src len, hid dim]
            
            attention = self.v(energy).squeeze(2)
            #attention = [batch size, src len]
            
        elif self.variants == 'general': #work
            hidden = hidden.unsqueeze(1).repeat(1, 1, 2)
            #hidden = [batch size, 1, hid dim*2]
            #encoder_outputs = [batch size, hid dim * 2, src len]

            energy = torch.bmm(hidden, encoder_outputs.transpose(1, 2))
            attention = energy.squeeze(1)
            #attention = [batch size, src len]

        elif self.variants == 'multiplicative':
            wh = self.W(hidden).unsqueeze(1).repeat(1, 1, 2)
            #wh = [batch size, 1, hid dim*2]
            #encoder_outputs = [batch size, hid dim * 2, src len]

            energy = torch.bmm(wh, encoder_outputs.transpose(1, 2))
            attention = energy.squeeze(1)

        #use masked_fill_ if you want in-place
        attention = attention.masked_fill(mask, -1e10)
        #attention = [batch size, src len]
        return F.softmax(attention, dim = 1)

Let's see how the masked_fill work, just so you have faith

In [45]:
#example of masked_fill
#reall that 1 is pad_idx
x = torch.tensor([ [9, 1, 7, 2, 1, 1], [99, 1, 1, 0, 8, 9] ])

mask = (x == PAD_IDX)

x.masked_fill_(mask, -1e10)
print(x)

tensor([[           9, -10000000000,            7,            2, -10000000000,
         -10000000000],
        [          99, -10000000000, -10000000000,            0,            8,
                    9]])


### Decoder

The decoder only needs a few small changes. It needs to accept a mask over the source sentence and pass this to the attention module. As we want to view the values of attention during inference, we also return the attention tensor.

In [46]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU((hid_dim * 2) + emb_dim, hid_dim)
        self.fc = nn.Linear((hid_dim * 2) + hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
        
        #input = [batch size]
        #hidden = [batch size, hid dim]
        #encoder_outputs = [src len, batch size, hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
        #a = [batch size, src len]

        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs) #Ct
        #weighted = [batch size, 1, hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, (hid dim * 2) + emb dim]
        
        output, hidden = self.gru(rnn_input, hidden.unsqueeze(0))
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [1, batch size, hid dim]
        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output   = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc(torch.cat((output, weighted, embedded), dim = 1))
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Putting them together (become Seq2Seq!)

The overarching seq2seq model also needs a few changes for packed padded sequences, masking and inference. 

We need to tell it what the indexes are for the pad token and also pass the source sentence lengths as input to the `forward` method.

We use the pad token index to create the masks, by creating a mask tensor that is 1 wherever the source sentence is not equal to the pad token. This is all done within the `create_mask` function.

The sequence lengths as needed to pass to the encoder to use packed padded sequences.

The attention at each time-step is stored in the `attentions` 

In [47]:
class Seq2SeqPackedAttention(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        #src: [src len, batch size]
        
        mask = (src == self.src_pad_idx).permute(1, 0)
        #mask: [batch size, src len]
        #we need to permute to make the mask same shape as attention...
        return mask

        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5,is_inference = False):
        if is_inference:
            batch_size = src.shape[1]
 
            trg_vocab_size = self.decoder.output_dim #define in decoder
            src_len_ = src.shape[0]
            #because decoder decodes each step....let's create a list that gonna append the result to this guy
            outputs = torch.zeros(src_len_, batch_size, trg_vocab_size).to(self.device)
            
            #because decoder decodes each step....let's memorize the attention done in each step....
            attentions = torch.zeros(src_len_, batch_size, src.shape[0]).to(self.device)
            
            #let's start!!!
            #1. encoder
            encoder_outputs, hidden = self.encoder(src, src_len)
            #encoder_outputs: [src len, batch size, hid dim * num directions]
            #hidden: [batch size, hid dim]
            
            #set the first input to the decoder
            input_ = src[0,:] #trg[0,:]  #basically <sos>
            
            #create the mask for use in this step
            mask = self.create_mask(src)
            
            #2. for each of trg word
            for t in range(1, src_len_):

                #3. decode (hidden is always carry forward)
                output, hidden, attention = self.decoder(input_, hidden, encoder_outputs, mask)
                #output:   [batch size, output_dim]
                #hidden:   [batch size, hid_dim]
                #attention::[batch size, src len]  ==> how each of src token is important to input_ 
                
                #4. append the results to outputs and attentions
                outputs[t] = output
                attentions[t] = attention
                
                #5. get the result, using argmax
                input_ = output.argmax(1)  #find the maximum index of dimension 1, which is output_dim
                
                #6. based on the teacher forcing ratio, 
                # teacher_force_or_not = random.random() < teacher_forcing_ratio
                # if trg_len is not None:
                #     trg_len    = trg.shape[0]
                #         #if teacher forcing, next input is the next trg
                #         #if no teacher forcing, the next input is the argmax guy...
                # input_ = trg[t] if teacher_force_or_not else top1  #autoregressive
                
            return outputs, attentions #outputs for predicting the word, attentions to see which word is important

        #src: [src len, batch size]
        #src len: [batch size]
        #trg: [trg len, batch size]
    
        else:
            batch_size = src.shape[1]
            trg_len    = trg.shape[0]
            trg_vocab_size = self.decoder.output_dim #define in decoder
            
            #because decoder decodes each step....let's create a list that gonna append the result to this guy
            outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
            
            #because decoder decodes each step....let's memorize the attention done in each step....
            attentions = torch.zeros(trg_len, batch_size, src.shape[0]).to(self.device)
            
            #let's start!!!
            #1. encoder
            encoder_outputs, hidden = self.encoder(src, src_len)
            #encoder_outputs: [src len, batch size, hid dim * num directions]
            #hidden: [batch size, hid dim]
            
            #set the first input to the decoder
            input_ = trg[0,:]  #basically <sos>
            
            #create the mask for use in this step
            mask = self.create_mask(src)
            
            #2. for each of trg word
            for t in range(1, trg_len):

                #3. decode (hidden is always carry forward)
                output, hidden, attention = self.decoder(input_, hidden, encoder_outputs, mask)
                #output:   [batch size, output_dim]
                #hidden:   [batch size, hid_dim]
                #attention::[batch size, src len]  ==> how each of src token is important to input_ 
                
                #4. append the results to outputs and attentions
                outputs[t] = output
                attentions[t] = attention
                
                #5. get the result, using argmax
                top1 = output.argmax(1)  #find the maximum index of dimension 1, which is output_dim
                
                #6. based on the teacher forcing ratio, 
                teacher_force_or_not = random.random() < teacher_forcing_ratio
                        #if teacher forcing, next input is the next trg
                        #if no teacher forcing, the next input is the argmax guy...
                input_ = trg[t] if teacher_force_or_not else top1  #autoregressive
                
            return outputs, attentions #outputs for predicting the word, attentions to see which word is important
        
        

## 6. Training

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [48]:
def initialize_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

### 6.1 General Attention

In [49]:
input_dim   = len(vocab_transform[SRC_LANGUAGE])
output_dim  = len(vocab_transform[TRG_LANGUAGE])
emb_dim     = 256  
hid_dim     = 512  
dropout     = 0.5
SRC_PAD_IDX = PAD_IDX

attn = Attention(hid_dim, variants='general')
enc  = Encoder(input_dim,  emb_dim,  hid_dim, dropout)
dec  = Decoder(output_dim, emb_dim,  hid_dim, dropout, attn)

model_general = Seq2SeqPackedAttention(enc, dec, SRC_PAD_IDX, device).to(device)
model_general.apply(initialize_weights)

Seq2SeqPackedAttention(
  (encoder): Encoder(
    (embedding): Embedding(1298, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (v): Linear(in_features=512, out_features=1, bias=False)
      (W): Linear(in_features=512, out_features=512, bias=True)
      (U): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(1796, 256)
    (gru): GRU(1280, 512)
    (fc): Linear(in_features=1792, out_features=1796, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [50]:
#we can print the complexity by the number of parameters
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model_general)

332288
393216
786432
  1536
  1536
393216
786432
  1536
  1536
524288
   512
   512
262144
   512
524288
   512
459776
1966080
786432
  1536
  1536
3218432
  1796
______
10446084


Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [51]:
import torch.optim as optim

lr = 0.001

#training hyperparameters
optimizer = optim.Adam(model_general.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX) #combine softmax with cross entropy

In [52]:
def train(model, loader, optimizer, criterion, clip, loader_length):
    
    model.train()
    epoch_loss = 0
    
    for src, src_length, trg in loader:
        
        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        output, attentions = model(src, src_length, trg)
        
        #trg    = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        #the loss function only works on 2d inputs with 1d targets thus we need to flatten each of them
        output = output[1:].view(-1, output_dim)
        trg    = trg[1:].view(-1)
        #trg    = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        #clip the gradients to prevent them from exploding (a common issue in RNNs)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / loader_length

In [53]:
def evaluate(model, loader, criterion, loader_length):
        
    #turn off dropout (and batch norm if used)
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for src, src_length, trg in loader:
        
            src = src.to(device)
            trg = trg.to(device)

            output, attentions = model(src, src_length, trg, 0) #turn off teacher forcing

            #trg    = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg    = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / loader_length

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
best_valid_loss = float('inf')
num_epochs = 5
clip       = 1

save_path = f'models/{model_general.__class__.__name__}_general.pt'

train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    
    start_time = time.time()

    train_loss = train(model_general, train_loader, optimizer, criterion, clip, train_loader_length)
    valid_loss = evaluate(model_general, valid_loader, criterion, val_loader_length)
    
    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_general.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    #lower perplexity is better

Epoch: 01 | Time: 2m 5s
	Train Loss: 5.763 | Train PPL: 318.200
	 Val. Loss: 5.329 |  Val. PPL: 206.328
Epoch: 02 | Time: 2m 6s
	Train Loss: 5.256 | Train PPL: 191.808
	 Val. Loss: 5.261 |  Val. PPL: 192.731
Epoch: 03 | Time: 2m 11s
	Train Loss: 4.984 | Train PPL: 146.005
	 Val. Loss: 5.084 |  Val. PPL: 161.489
Epoch: 04 | Time: 2m 3s
	Train Loss: 4.667 | Train PPL: 106.418
	 Val. Loss: 4.914 |  Val. PPL: 136.179
Epoch: 05 | Time: 2m 10s
	Train Loss: 4.346 | Train PPL:  77.138
	 Val. Loss: 4.869 |  Val. PPL: 130.241


### 6.2 Multiplicative Attention

In [92]:
input_dim   = len(vocab_transform[SRC_LANGUAGE])
output_dim  = len(vocab_transform[TRG_LANGUAGE])
emb_dim     = 256  
hid_dim     = 512  
dropout     = 0.5
SRC_PAD_IDX = PAD_IDX

attn = Attention(hid_dim, variants='multiplicative')
enc  = Encoder(input_dim,  emb_dim,  hid_dim, dropout)
dec  = Decoder(output_dim, emb_dim,  hid_dim, dropout, attn)

model_multi = Seq2SeqPackedAttention(enc, dec, SRC_PAD_IDX, device).to(device)
model_multi.apply(initialize_weights)

Seq2SeqPackedAttention(
  (encoder): Encoder(
    (embedding): Embedding(1298, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (v): Linear(in_features=512, out_features=1, bias=False)
      (W): Linear(in_features=512, out_features=512, bias=True)
      (U): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(1796, 256)
    (gru): GRU(1280, 512)
    (fc): Linear(in_features=1792, out_features=1796, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [93]:
#we can print the complexity by the number of parameters
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model_multi)

332288
393216
786432
  1536
  1536
393216
786432
  1536
  1536
524288
   512
   512
262144
   512
524288
   512
459776
1966080
786432
  1536
  1536
3218432
  1796
______
10446084


In [94]:
import torch.optim as optim

lr = 0.001

#training hyperparameters
optimizer = optim.Adam(model_multi.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX) #combine softmax with cross entropy

In [95]:
def train(model, loader, optimizer, criterion, clip, loader_length):
    
    model.train()
    epoch_loss = 0
    
    for src, src_length, trg in loader:
        
        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        output, attentions = model(src, src_length, trg)
        
        #trg    = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        #the loss function only works on 2d inputs with 1d targets thus we need to flatten each of them
        output = output[1:].view(-1, output_dim)
        trg    = trg[1:].view(-1)
        #trg    = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        #clip the gradients to prevent them from exploding (a common issue in RNNs)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / loader_length

Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

In [96]:
def evaluate(model, loader, criterion, loader_length):
        
    #turn off dropout (and batch norm if used)
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for src, src_length, trg in loader:
        
            src = src.to(device)
            trg = trg.to(device)

            output, attentions = model(src, src_length, trg, 0) #turn off teacher forcing

            #trg    = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg    = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / loader_length

#### Putting everything together

In [97]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [98]:
best_valid_loss = float('inf')
num_epochs = 1
clip       = 1

save_path = f'models/{model_multi.__class__.__name__}_multiplicative.pt'

train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    
    start_time = time.time()

    train_loss = train(model_multi, train_loader, optimizer, criterion, clip, train_loader_length)
    valid_loss = evaluate(model_multi, valid_loader, criterion, val_loader_length)
    
    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_multi.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    #lower perplexity is better

Epoch: 01 | Time: 2m 30s
	Train Loss: 5.767 | Train PPL: 319.733
	 Val. Loss: 5.449 |  Val. PPL: 232.572


In [121]:
# model_multi.load_state_dict(torch.load(save_path))
# test_loss = evaluate(model_multi, test_loader, criterion, test_loader_length)

# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.757 | Test PPL: 116.408 |


### 6.3 Additive Attention

In [99]:
input_dim   = len(vocab_transform[SRC_LANGUAGE])
output_dim  = len(vocab_transform[TRG_LANGUAGE])
emb_dim     = 256  
hid_dim     = 512  
dropout     = 0.5
SRC_PAD_IDX = PAD_IDX

attn = Attention(hid_dim, variants='additive')
enc  = Encoder(input_dim,  emb_dim,  hid_dim, dropout)
dec  = Decoder(output_dim, emb_dim,  hid_dim, dropout, attn)

model_additive = Seq2SeqPackedAttention(enc, dec, SRC_PAD_IDX, device).to(device)
model_additive.apply(initialize_weights)

Seq2SeqPackedAttention(
  (encoder): Encoder(
    (embedding): Embedding(1298, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (v): Linear(in_features=512, out_features=1, bias=False)
      (W): Linear(in_features=512, out_features=512, bias=True)
      (U): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(1796, 256)
    (gru): GRU(1280, 512)
    (fc): Linear(in_features=1792, out_features=1796, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [100]:
#we can print the complexity by the number of parameters
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model_additive)

332288
393216
786432
  1536
  1536
393216
786432
  1536
  1536
524288
   512
   512
262144
   512
524288
   512
459776
1966080
786432
  1536
  1536
3218432
  1796
______
10446084


Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [101]:
import torch.optim as optim

lr = 0.001

#training hyperparameters
optimizer = optim.Adam(model_additive.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX) #combine softmax with cross entropy

The training is very similar to part 1.

In [102]:
def train(model, loader, optimizer, criterion, clip, loader_length):
    
    model.train()
    epoch_loss = 0
    
    for src, src_length, trg in loader:
        
        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        output, attentions = model(src, src_length, trg)
        
        #trg    = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        #the loss function only works on 2d inputs with 1d targets thus we need to flatten each of them
        output = output[1:].view(-1, output_dim)
        trg    = trg[1:].view(-1)
        #trg    = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        #clip the gradients to prevent them from exploding (a common issue in RNNs)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / loader_length

Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

In [103]:
def evaluate(model, loader, criterion, loader_length):
        
    #turn off dropout (and batch norm if used)
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for src, src_length, trg in loader:
        
            src = src.to(device)
            trg = trg.to(device)

            output, attentions = model(src, src_length, trg, 0) #turn off teacher forcing

            #trg    = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg    = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / loader_length

#### Putting everything together

In [104]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [105]:
best_valid_loss = float('inf')
num_epochs = 1
clip       = 1

save_path = f'models/{model_additive.__class__.__name__}_additive.pt'

train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    
    start_time = time.time()

    train_loss = train(model_additive, train_loader, optimizer, criterion, clip, train_loader_length)
    valid_loss = evaluate(model_additive, valid_loader, criterion, val_loader_length)
    
    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_additive.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    #lower perplexity is better

Epoch: 01 | Time: 3m 37s
	Train Loss: 5.740 | Train PPL: 311.140
	 Val. Loss: 5.257 |  Val. PPL: 191.814


In [106]:
model_additive.load_state_dict(torch.load(save_path))
test_loss = evaluate(model_additive, test_loader, criterion, test_loader_length)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.318 | Test PPL: 204.055 |


### 6.4 Conclusion
> I did not have any resource to run this model so I will compare with a thing that good for me which is  a training time

- **General Attetion** training time is 2.05 mins per epoch which mean it quite effieient cpmpare to other attentoin

- **Multiplicative Attetion** training time is 3.11 mins per epoch which mean it heavier cpmpare to general attentoin

- **Additive Attetion** training time is 3.37 mins per epoch which mean it quite Heavy cpmpare to General attention




## 7. Test on some random news

In [58]:
dataset[0][0], dataset[0][1]


('ฉันค่อนข้างมีความสุขกับหนังสือ', "I'm pretty happy with the book!.")

In [79]:
src_text = text_transform[SRC_LANGUAGE]('ฉันชอบมัน').to(device)
trg_text = text_transform[TRG_LANGUAGE]('another cable').to(device)
src_text, trg_text

(tensor([ 2,  6, 84,  8,  3]), tensor([  2, 110, 534,   3]))

In [80]:
src_text = src_text.reshape(-1, 1)  #because batch_size is 1
trg_text = trg_text.reshape(-1, 1)
src_text.shape, trg_text.shape

(torch.Size([5, 1]), torch.Size([4, 1]))

In [81]:
text_length = torch.tensor([src_text.size(0)]).to(dtype=torch.int64)

In [82]:
model_additive = model_general

In [83]:
model_additive.load_state_dict(torch.load(save_path))

model_additive.eval()
with torch.no_grad():
    output, attentions = model_additive(src_text, text_length, None, 0,True) #turn off teacher forcing

In [84]:
output.shape #trg_len, batch_size, trg_output_dim

torch.Size([5, 1, 1796])

Since batch size is 1, we just take off that dimension

In [85]:
output = output.squeeze(1)

In [86]:
output.shape

torch.Size([5, 1796])

We shall remove the first token since it's zeroes anyway

In [87]:
output = output[1:]
output.shape #trg_len, trg_output_dim

torch.Size([4, 1796])

Then we just take the top token with highest probabilities

In [88]:
output_max = output.argmax(1) #returns max indices

In [89]:
output_max

tensor([  7, 196,   8,   6])

Get the mapping of the target language

In [90]:
mapping = vocab_transform[TRG_LANGUAGE].get_itos()

In [91]:
for token in output_max:
    print(mapping[token.item()])

I
love
the
it
